In [ ]:
from typing import NamedTuple

class ANamedTuple(NamedTuple):
    """a docstring"""
    foo: int
    bar: str
    baz: list

a = ANamedTuple(foo=1, bar="asd", baz=[2])
foo, bar, baz = a


In [2]:
import numpy as np


In [5]:
sub_degree = 30
fixed_assignments = []
randomly_select = None
exclude_inverse = True
exclude_isolations = True


sub_degree -= len(fixed_assignments)

# -1 because we concatenate the inverse only on demand.
num_possible_splits = 2 ** (sub_degree - 1)
if randomly_select is not None:
    randomly_select = min(randomly_select, num_possible_splits)
    action_range = np.random.choice(int(num_possible_splits), randomly_select)
    # We can't pass replace=False to np.random.choice because that would implement a massive
    # array internally, hence we just filter out duplicates here
    action_range = np.unique(action_range)
else:
    action_range = np.arange(num_possible_splits)

# The repo is just a binary representation of the numbers from 0 to num_possible_splits
# However, we don't want to include the inverse of the actions, hence we only need to count
# up to num_possible_splits / 2.
repo = np.zeros((len(action_range), sub_degree), dtype=bool)
for i in range(sub_degree):
    repo[:, i] = np.bitwise_and(np.right_shift(action_range, i), 1)

# We only want to keep the inverse that has fewer elements on busbar B
# Hence, we invert where there are more than sub_degree / 2 elements on busbar B
# If exclude_inverse is False, we also include the inverse by just concatenating them to the
# repo
if exclude_inverse:
    inverse_is_better = np.sum(repo, axis=1) > (sub_degree / 2)
    repo[inverse_is_better, :] = ~repo[inverse_is_better, :]
else:
    repo = np.concatenate([repo, ~repo], axis=0)

# We also exclude all actions that isolate a branch if that's desired
if exclude_isolations:
    num_bus_b = np.sum(repo, axis=1)
    repo = repo[(num_bus_b != 1) & (num_bus_b != sub_degree - 1), :]

# If there are fixed assignments, we need to apply them
for branch_idx, busbar_b in fixed_assignments:
    repo = np.insert(repo, branch_idx, busbar_b, axis=1)

# Make sure the first combination is the unsplit action
# The fixed assignments might have changed this
repo[0, :] = False


: 

In [1]:
import numpy as np

In [4]:
np.random.choice(2**25, size=2**20, replace=False)

array([16803256, 17495681, 29390505, ..., 13759324,  8297533, 32440531])

In [93]:
repo.shape

(16369, 15)

In [3]:

import itertools
from functools import lru_cache
from dataclasses import dataclass

import jax.numpy as jnp
import logbook
import networkx as nx
import numpy as np
from beartype.typing import Literal, NamedTuple, Optional, Sequence
from jaxtyping import Array, Bool, Int
from networkx.algorithms.components import (
    connected_components,
    number_connected_components,
)
from toop_engine_dc_solver.preprocess.helpers.switching_distance import hamming_distance
from toop_engine_interfaces.asset_topology import (
    Busbar,
    BusbarCoupler,
    Station,
    SwitchableAsset,
    Topology,
)
from toop_engine_interfaces.asset_topology_helpers import (
    filter_disconnected_busbars,
    filter_duplicate_couplers,
    filter_out_of_service,
    fix_multi_connected_without_coupler,
    fuse_all_couplers_with_type,
    order_station_assets,
    order_topology,
)


In [42]:
sub_degree =19
max_reassignment = 4
randomly_select =None
# -1 because we concatenate the inverse only on demand.
num_possible_splits = 2 ** (sub_degree - 1)
if randomly_select is not None:
    randomly_select = min(randomly_select, num_possible_splits)
    action_range = np.random.choice(int(num_possible_splits), randomly_select)
    # We can't pass replace=False to np.random.choice because that would implement a massive
    # array internally, hence we just filter out duplicates here
    action_range = np.unique(action_range)
else:
    action_range = np.arange(num_possible_splits)
repo = np.zeros((len(action_range), sub_degree), dtype=bool)
for i in range(sub_degree):
    repo[:, i] = np.bitwise_and(np.right_shift(action_range, i), 1)

keep_indices = np.ones(repo.shape[0], dtype=bool)
for setups in configurations:
    for config in setups:
        start_time = time.time()

        keep_indices &= (config != repo).sum(axis=1) < max_reassignment
        end_time = time.time()
        print(f"Config {config} took {end_time - start_time} seconds and has")


Config [False  True  True False False  True False  True  True False  True False
 False  True False  True  True False  True] took 0.012095928192138672 seconds and has
Config [ True False False  True False  True  True False  True False False  True
 False  True  True False  True False  True] took 0.013066768646240234 seconds and has
Config [ True  True  True False False  True False False  True  True  True  True
 False  True  True  True  True  True  True] took 0.013512372970581055 seconds and has
Config [False  True False False  True False  True  True False  True False  True
  True False False False  True False False] took 0.012965679168701172 seconds and has


In [46]:
((config != repo).sum(axis=1) == 4).sum()

3060

In [48]:
((config != repo).sum(axis=1) < max_reassignment).sum()

988

In [37]:
config

array([False,  True, False, False,  True, False,  True,  True, False,
        True, False,  True,  True, False, False, False,  True, False,
       False])

In [40]:
repo

array([[False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [ True, False,  True, ...,  True,  True, False],
       [False,  True,  True, ...,  True,  True, False],
       [ True,  True,  True, ...,  True,  True, False]])

In [39]:
(config != repo)

array([[False,  True, False, ...,  True, False, False],
       [ True,  True, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [ True,  True,  True, ..., False,  True, False],
       [False, False,  True, ..., False,  True, False],
       [ True, False,  True, ..., False,  True, False]])

In [34]:
(config == repo).sum(axis=1).min()

1

In [36]:
keep_indices.sum()

0

In [26]:
configurations.shape

(2, 2, 19)

In [23]:
repo.shape

(524288, 20)

In [14]:
for setups in configurations:
    for config in setups:
        config == repo

ValueError: operands could not be broadcast together with shapes (19,) (662848,21) 

In [11]:
repo == config

ValueError: operands could not be broadcast together with shapes (662848,21) (2,19) 

In [5]:
@lru_cache(maxsize=None)
def _get_flip_masks(n_assets: int, max_flips: int) -> Bool[np.ndarray, " n_permutations n_assets"]:
    """Precompute boolean flip masks up to ``max_flips`` for ``n_assets`` assets."""

    base_mask = np.zeros((1, n_assets), dtype=bool)
    if max_flips == 0 or n_assets == 0:
        return base_mask

    masks = [base_mask]
    asset_indices = np.arange(n_assets)
    for n_flips in range(1, max_flips + 1):
        combinations = np.array(list(itertools.combinations(asset_indices, n_flips)), dtype=int)
        if combinations.size == 0:
            continue
        mask = np.zeros((combinations.shape[0], n_assets), dtype=bool)
        mask[np.arange(combinations.shape[0])[:, None], combinations] = True
        masks.append(mask)

    return np.concatenate(masks, axis=0)
def enumerate_switching_reassignments(
    configurations: Bool[np.ndarray, " n_configurations 2 n_assets"],
    max_reassignments: int,
) -> Bool[np.ndarray, " n_permutations n_assets"]:
    """Enumerate all asset reassignments up to a maximum number of switches.

    Parameters
    ----------
    configurations : Bool[np.ndarray, " n_configurations 2 n_assets"]
        The base switching configurations where the second row captures the secondary busbar state.
    max_reassignments : int
        The maximum number of asset reassignments allowed per configuration. Must be non-negative.

    Returns
    -------
    Bool[np.ndarray, " n_permutations n_assets"]
        All reachable secondary busbar states aggregated across the provided configurations with up
        to ``max_reassignments`` asset flips. The number of rows corresponds to the total number of
        generated states.

    Raises
    ------
    ValueError
        If ``max_reassignments`` is negative or the configurations do not expose exactly two
        busbar layers.
    """

    if max_reassignments < 0:
        raise ValueError("max_reassignments must be non-negative.")
    if configurations.ndim != 3 or configurations.shape[1] != 2:
        raise ValueError(
            "Configurations must have shape (n_configurations, 2, n_assets) with primary and secondary layers."
        )

    _, _, n_assets = configurations.shape
    max_flips = min(max_reassignments, n_assets)
    if max_flips == 0:
        return np.array(configurations[:, 1, :], dtype=bool)

    masks = _get_flip_masks(n_assets=n_assets, max_flips=max_flips)
    permutations_per_configuration: list[np.ndarray] = []

    for configuration in configurations:
        base_secondary = configuration[1]
        permutations_per_configuration.append(np.logical_xor(base_secondary[None, :], masks))

    return np.concatenate(permutations_per_configuration, axis=0)

In [6]:
import time 

for sub_degree in range(4, 20):
    configurations = np.array(
        [
            [
                np.random.choice([True, False], size=(sub_degree,)),
                np.random.choice([True, False], size=(sub_degree,)),
            ],
            [
                np.random.choice([True, False], size=(sub_degree,)),
                np.random.choice([True, False], size=(sub_degree,)),
            ],
        ],
        dtype=bool,
    )
    for i in range(20):
        start_time = time.time()
        reassignments = enumerate_switching_reassignments(configurations, max_reassignments=i)
        end_time = time.time()
        print(f"Max reassignments: {i}, total reassignments: {len(reassignments)}, time taken: {end_time - start_time:.4f} seconds")

Max reassignments: 0, total reassignments: 2, time taken: 0.0000 seconds
Max reassignments: 1, total reassignments: 10, time taken: 0.0002 seconds
Max reassignments: 2, total reassignments: 22, time taken: 0.0005 seconds
Max reassignments: 3, total reassignments: 30, time taken: 0.0001 seconds
Max reassignments: 4, total reassignments: 32, time taken: 0.0001 seconds
Max reassignments: 5, total reassignments: 32, time taken: 0.0000 seconds
Max reassignments: 6, total reassignments: 32, time taken: 0.0000 seconds
Max reassignments: 7, total reassignments: 32, time taken: 0.0000 seconds
Max reassignments: 8, total reassignments: 32, time taken: 0.0000 seconds
Max reassignments: 9, total reassignments: 32, time taken: 0.0000 seconds
Max reassignments: 10, total reassignments: 32, time taken: 0.0000 seconds
Max reassignments: 11, total reassignments: 32, time taken: 0.0000 seconds
Max reassignments: 12, total reassignments: 32, time taken: 0.0000 seconds
Max reassignments: 13, total reassig

In [20]:
enumerate_switching_reassignments(configurations, max_reassignments=0)

array([[ True, False, False,  True,  True,  True,  True,  True, False,
        False],
       [False,  True,  True,  True, False, False, False, False, False,
        False]])

In [ ]:
reassignments ==

array([[False,  True, False, ..., False,  True, False],
       [ True,  True, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       ...,
       [ True,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])

In [ ]:
sub_degree = 30
fixed_assignments = []
randomly_select = 2 ** 50
50
exclude_inverse = True
exclude_isolations = True


sub_degree -= len(fixed_assignments)

# -1 because we concatenate the inverse only on demand.
num_possible_splits = 2 ** (sub_degree - 1)
if randomly_select is not None:
    randomly_select = min(randomly_select, num_possible_splits)
    action_range = np.random.choice(int(num_possible_splits), randomly_select)
    # We can't pass replace=False to np.random.choice because that would implement a massive
    # array internally, hence we just filter out duplicates here
    action_range = np.unique(action_range)
else:
    action_range = np.arange(num_possible_splits)

# The repo is just a binary representation of the numbers from 0 to num_possible_splits
# However, we don't want to include the inverse of the actions, hence we only need to count
# up to num_possible_splits / 2.
repo = np.zeros((len(action_range), sub_degree), dtype=bool)
for i in range(sub_degree):
    repo[:, i] = np.bitwise_and(np.right_shift(action_range, i), 1)


In [ ]:
sub_degree = 20
sub_degree -= len(fixed_assignments)
randomly_select = 2 ** 10
# -1 because we concatenate the inverse only on demand.
num_possible_splits = 2 ** (sub_degree - 1)
if randomly_select is not None:
    randomly_select = min(randomly_select, num_possible_splits)
    action_range = np.random.choice(int(num_possible_splits), randomly_select)
    # We can't pass replace=False to np.random.choice because that would implement a massive
    # array internally, hence we just filter out duplicates here
    action_range = np.unique(action_range)
else:
    action_range = np.arange(num_possible_splits)

# The repo is just a binary representation of the numbers from 0 to num_possible_splits
# However, we don't want to include the inverse of the actions, hence we only need to count
# up to num_possible_splits / 2.
repo = np.zeros((len(action_range), sub_degree), dtype=bool)
for i in range(sub_degree):
    repo[:, i] = np.bitwise_and(np.right_shift(action_range, i), 1)

# We only want to keep the inverse that has fewer elements on busbar B
# Hence, we invert where there are more than sub_degree / 2 elements on busbar B
# If exclude_inverse is False, we also include the inverse by just concatenating them to the
# repo
if exclude_inverse:
    inverse_is_better = np.sum(repo, axis=1) > (sub_degree / 2)
    repo[inverse_is_better, :] = ~repo[inverse_is_better, :]
else:
    repo = np.concatenate([repo, ~repo], axis=0)

# We also exclude all actions that isolate a branch if that's desired
if exclude_isolations:
    num_bus_b = np.sum(repo, axis=1)
    repo = repo[(num_bus_b != 1) & (num_bus_b != sub_degree - 1), :]

# If there are fixed assignments, we need to apply them
for branch_idx, busbar_b in fixed_assignments:
    repo = np.insert(repo, branch_idx, busbar_b, axis=1)

# Make sure the first combination is the unsplit action
# The fixed assignments might have changed this
repo[0, :] = False


1024

In [ ]:
import numpy as np
sub_degree = 20
randomly_select = None
# -1 because we concatenate the inverse only on demand.
num_possible_splits = 2 ** (sub_degree - 1)
if randomly_select is not None:
    randomly_select = min(randomly_select, num_possible_splits)
    action_range = np.random.choice(int(num_possible_splits), randomly_select)
    # We can't pass replace=False to np.random.choice because that would implement a massive
    # array internally, hence we just filter out duplicates here
    action_range = np.unique(action_range)
else:
    action_range = np.arange(num_possible_splits)

# The repo is just a binary representation of the numbers from 0 to num_possible_splits
# However, we don't want to include the inverse of the actions, hence we only need to count
# up to num_possible_splits / 2.
repo = np.zeros((len(action_range), sub_degree), dtype=bool)
for i in range(sub_degree):
    repo[:, i] = np.bitwise_and(np.right_shift(action_range, i), 1)

In [10]:
# remove row with all false values
repo = repo[np.any(repo, axis=1)]

In [57]:
configurations = np.array(
        [
            [
                [True, False, True],
                [False, True, False],
            ],
            [
                [False, False, False],
                [True, True, True],
            ],
        ],
        dtype=bool,
    )
configurations[:,1, :]

array([[False,  True, False],
       [ True,  True,  True]])

In [83]:
a = np.array(
    [
        [True, False, False],
        [False, False, False],
        [True, True, True],
        [True, True, False]
    ]
)

b = np.array(
    [
        [False, False, False],
    ]
)

In [89]:
getattr(None, "cool", "123")

'123'

In [87]:
a = np.array([1,6,3])
b = np.array([4,5,6])
np.min([a,b], axis=0)

array([1, 5, 3])

In [84]:
np.insert(a, slice(0, a.shape[1]), False, axis=1)

array([[False,  True, False, False, False, False],
       [False, False, False, False, False, False],
       [False,  True, False,  True, False,  True],
       [False,  True, False,  True, False, False]])

In [70]:
dist_matrix = np.sum(a[:, None, :] != b[None, :, :], axis=2)
min_changes = np.min(dist_matrix, axis=1)
min_changes

array([1, 0, 3, 2])

In [60]:
dist_matrix

array([[0, 2],
       [2, 0]])

In [56]:
from toop_engine_dc_solver.preprocess.action_set import make_action_repo
results = {}
for sub_degree in range(19, 30):
    results[sub_degree] = []
    for i in range(1000):
        action_repo = make_action_repo(sub_degree=sub_degree, fixed_assignments=(), randomly_select=2**20, exclude_inverse=True, exclude_isolations=True)
        res = action_repo.shape[0] / 2 ** 20
        results[sub_degree].append(res)
        print(res)

0.15800094604492188
0.1581573486328125
0.157958984375
0.15793132781982422
0.15799522399902344
0.1575765609741211
0.1581735610961914
0.15803146362304688
0.15789794921875
0.15798473358154297
0.15798091888427734
0.15795230865478516
0.15776443481445312
0.15794944763183594
0.15816688537597656
0.1580648422241211
0.15792179107666016
0.15795230865478516
0.15827178955078125
0.15812206268310547
0.15811824798583984
0.15827178955078125
0.15787220001220703
0.15797805786132812
0.1580982208251953
0.15812206268310547
0.15806007385253906
0.15791702270507812
0.15805816650390625
0.15819358825683594
0.15801429748535156
0.15808773040771484
0.15797901153564453
0.15823936462402344
0.15784358978271484
0.15789318084716797
0.15830135345458984
0.1580982208251953
0.15802860260009766
0.15819644927978516
0.1578540802001953
0.1578836441040039
0.15807247161865234
0.15793991088867188
0.1581277847290039
0.15803050994873047
0.15795421600341797
0.15779972076416016
0.15766525268554688
0.15794086456298828
0.158032417297363

KeyboardInterrupt: 

In [42]:
len(repo)

1024

In [33]:
randomly_select = min(randomly_select, num_possible_splits)
action_range = np.random.choice(int(num_possible_splits), randomly_select)

In [35]:
action_range = np.unique(action_range)

In [37]:
action_range

array([     0,      1,      2, ..., 524284, 524285, 524287])

In [24]:
repo[5]

array([ True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [27]:
np.sum(repo != repo[5], axis=1).max()

14

In [1]:
repo[0, :] = False


NameError: name 'repo' is not defined